In a laptop, iteratively calling Eclipse to perform flow simulation for each group of permeability maps and boundary pressure values

Steps:

1. Construct an Eclipse flow simulation case, referred to as "Case5" here.

2. For each set of permeability map and boundary pressure values, insert them and replace the corresponding data in Case5.

3. Run Case5 using Eclipse.

4. Collect the simulated pressure maps and copy them to a separate folder.

In [ ]:
import os
import numpy as np
import shutil

# Path to the Eclipse executable
eclrun = r'C:\ecl\macros\eclrun.exe'

# Directory to the constructed Eclipse case
simcasepath = r'F:\Papers\Papers\Paper7_FlowSimRelated\FlowSimulation\EclipseFlowSimCase_64x64_30day\Case5\\'

# Directory containing permeability and pressure input files
perm_pres_path = r'F:\Papers\Papers\Paper7_FlowSimRelated\FlowSimulation\PointBar_Flow_case1\Perm_BoundaryPres\\'

# Directory to save simulated pressure maps
sim_pres_path = r'F:\Papers\Papers\Paper7_FlowSimRelated\FlowSimulation\PointBar_Flow_case1\SimulatedPres\\'


In [ ]:
# Read press boundary values

presfile = perm_pres_path + 'PressureBoundaryValues.txt'
allele=[] 
with open(presfile, 'r') as f:
    for line in f:
        eles = line.strip().split('  ')
        allele.append(eles)
pressbound_values = np.array(allele).reshape((-1, 2)).astype(np.float32)

In [ ]:
# Write boundary pressure values and initial pressure value into _init.INC file of Eclipse Simulate Case

def writepress(press_array, row, init_file):
    [large_bound, small_bound] = press_array[row]
    os.remove(init_file)
    content = 'ECHO\n'+'EQUIL\n'
    content = content + '    3005     ' + str(small_bound) + '       0       0      1*       0       0       0       0      1*      1*'
    content = content + '/\n'+'AQUALIST\n'+'   QUIFER_1        2 /\n'+ '   TAQUIFER        1 /\n' + '/\n'+ 'AQUFETP\n'
    content = content + ' 1 3005 ' + str(large_bound) + '          1E+14 0.00014504 1.0E+14 1 0.0 /\n'
    content = content + ' 2 3005 ' + str(small_bound) + '          1E+14 0.00014504 1.0E+14 1 0.0 /\n'
    content = content + '/\n' + 'AQUANCON\n' + '     1     1    64     1     1     1     1 \'J+\'          1*          1* \'YES\'  /\n'
    content = content + '     2     1    64    64    64     1     1 \'J+\'          1*          1* \'YES\'  /\n'+'/\n'
    file = open(init_file, 'a')
    file.write(content)
    file.close()

In [ ]:
# function to collect simulated pressure maps and write them into new file
# used for reservoir dimensions larger than 15

def arrangepressuremaps(simcasepath, savepath, perm_num):
    D = 64
    a = D//15+1
    time_steps = 30
    
    pressmaparray_1 = np.zeros((D*a*time_steps, 15))
    array_row = 0
    resultfile = simcasepath + 'CASE5_E100.PRT'
    with open(resultfile, 'r') as f:
        for line in f:
            if line.startswith(' (*, '):
                s = line.strip()[13:]
                s = np.array(s.split(' ')).astype(np.float32)
                pressmaparray_1[array_row,:s.shape[0]] = s
                array_row += 1
    
    pressmaparray_2 = np.zeros((D*time_steps, D))
    for i in range(time_steps):
        for j in range(a-1):
            pressmaparray_2[i*D:(i+1)*D,j*15:(j+1)*15] = pressmaparray_1[i*D*a + j*D: i*D*a + (j+1)*D]
        pressmaparray_2[i*D:(i+1)*D,D-a+1:] = pressmaparray_1[i*D*a + (a-1)*D:i*D*a + a*D, 0:a-1]
    
    result_name = savepath+'sim_pressuremaps_'+str(perm_num)
    np.savetxt(result_name, pressmaparray_2, fmt="%1.3f", newline='\n')   # "%03d" + "%.10f"*7

In [ ]:
import time
import os

# Start overall timer
start_time = time.time()

for i in range(1, 202):
    iteration_start_time = time.time()  # Start timer for the current iteration

    # remove the original permability maps
    os.remove(simcasepath+'PermI.txt')
    # copy the target permeability maps
    shutil.copy2(perm_pres_path + 'permmap_' + str(i), simcasepath+'PermI.txt')
    # write the target boundary and initial pressure values
    writepress(pressbound_values, i-1, simcasepath + 'Case5_init.INC')   
    # run simulation with target permeability and pressure values
    os.system('eclrun eclipse ' + simcasepath + 'CASE5_E100.DATA')
    # collect the simulated pressure maps and save it into new folder
    arrangepressuremaps(simcasepath, sim_pres_path, i)

# Calculate the total time taken for the entire loop
end_time = time.time()
total_duration = end_time - start_time
print(f"Total time for the process: {total_duration:.2f} seconds")
    

In [ ]:
# method to read the simulated pressure maps

press_test_file = sim_pres_path + 'sim_pressuremaps_51'
allele=[] 
with open(press_test_file, 'r') as f:
    for line in f:
        eles = line.strip().split(' ')
        allele.append(eles)
pressmap_test = np.array(allele).reshape(-1,64,64).astype(np.float32)